## Formulating commandline for registration through elastix

Works for both sample2ara and ara2sample, just switch the 2 image file accordingly

Currently still need to manually run in cmd

Requirements: *mhd* formats of your image stack and allen brain template in the correct z orientation (can be done in imageJ to flip the Y order. Image->transform->flipz)

User inputs: define output directory, moving image, fixed image, elastix parameter file (in order)

In [1]:
import os

import subprocess

import tkinter.filedialog as fdialog

# this is the gui for finding directory and files

import shutil

First, we register allen template to our data (hence, ARA2Sample)
- fixed image= our data
- moving image= allen

Then, we use the same transform parameter files to the allen atlas
(affine first and then bspline provided by robcampbell UC in Han et al,2014)

The following works also for sample2ara by redefining the fixed and moving image
- fixedimage=allen
- moving image=data

In this case we need to transform an inverse transform and apply the inverse transform to our annotation

In [2]:
tempdir = fdialog.askdirectory(title='Please select a output directory')

In [3]:
fixed_img=fdialog.askopenfile(initialdir=tempdir, title='select the fixed image').name
# need the .name part because askopenfile returns an io.textiowrapper not the full str name
moving_img=fdialog.askopenfile(initialdir=tempdir, title='select the moving image').name


In [4]:
#Start with the transform parameters in the backup folder. If things don't work then switch to the working folder for adjustments.
param1=fdialog.askopenfile(initialdir=tempdir, title='select the first parameter file', initialdir = "D:/elastix_params_backup").name
param2=fdialog.askopenfile(initialdir=tempdir, title='select the second parameter file',initialdir = "D:/elastix_params_backup").name

In [5]:
shutil.copy(param1, tempdir)
shutil.copy(param2, tempdir)

#copy the 2 parameter files used to the output directory as backup

'D:/GF236/sample2ara\\02_ARA_bspline.txt'

In [5]:
command_line= ['elastix -f '+ fixed_img+ ' -m ' + moving_img+ 
               ' -out '+ tempdir+ ' -p '+ param1+
              ' -p ' + param2]
print('formulated commandline: ' , command_line)

#this step typically takes 10-20min depending on how stringent the criterias are

formulated commandline:  ['elastix -f D:/AL066/template118-440.mhd -m D:/AL066/AL066_25.0um.mhd -out D:/AL066 -p D:/elastix_params_backup/sample2ara/01_ARA_affine.txt -p D:/elastix_params_backup/sample2ara/02_ARA_bspline.txt']


open cmd, cd into the folder containing elastix.exe
Copy the entire thing inside quotation marks and run in cmd

To do: run this commandline in cmd through python


In [16]:
command=['elastix', 
         '-f', fixed_img, 
         '-m', moving_img,
         '-out', tempdir,
         '-p', param1, 
         '-p', param2
        ]

In [17]:
subprocess.run(command, cwd= 'C:/Users/liu')

FileNotFoundError: [WinError 2] The system cannot find the file specified

## For ARA2Sample: Transform the atlas
Now, we transform the atlas using the transform parameter values we got from the previous step

first need to chain the two parameter files (IntialTransformParametersFile )
Here, transformparam0 which has the affine values change (IntialTransformParametersFileName "NoInitialTransform")
and, transformparam1 which has the bspline values change (IntialTransformParametersFileName "insert_fullpathto_transformparam0")

then in transformparam1 change (FinalBSplineInterpolationOrder 0) and (ResultImagePixelType "int")
This is to stop interpolation (ie, for binary or masks images where the index is important to keep) and make resulting image type int 


finally use transformix 

In [2]:
tempdir = fdialog.askdirectory(title='Please select a output directory')

In [3]:
transparam0=fdialog.askopenfile(initialdir=tempdir, title='select the transformparameter0 file').name
transparam1=fdialog.askopenfile(initialdir=tempdir, title='select the transformparameter1 file').name
atlasname=fdialog.askopenfile(initialdir=tempdir, title='select the corresponding atlas file').name

In [5]:
# make the changes in transform parameters affine 
tparam0=open(transparam0,'r')
tparam0_data=tparam0.readlines()

# tparam0_data
# looks like (IntialTransformParametersFileName "NoInitialTransform") is already taken care of by elastix
# only need to change transparam1

In [6]:
# make the changes in transform parameters bspline

tparam1=open(transparam1,'r')
tparam1_data=tparam1.readlines()
tparam1.close()


new_transparam1_name=tempdir+'/tx_transparam1.txt'
new_tparam1=open(new_transparam1_name,'w+')

for lines in tparam1_data:
    if 'FinalBSplineInterpolationOrder' in lines:
        print(lines)
        print('Changing FinalBSplineInterpolationOrder 3 to 0')
        print(' ')
        new_line=lines.replace('3','0')
        new_tparam1.write(new_line)
    elif 'ResultImagePixelType ' in lines:
        print(lines)
        print('Changing ResultImagePixelType to int')
        new_line1=lines.replace('short', 'int')
        new_tparam1.write(new_line1)
    else:
        new_tparam1.write(lines)
new_tparam1.close()


(FinalBSplineInterpolationOrder 3)

Changing FinalBSplineInterpolationOrder 3 to 0
 
(ResultImagePixelType "short")

Changing ResultImagePixelType to int


In [7]:
command_line_transformix=['transformix -out '+ tempdir+ ' -tp ' + new_transparam1_name+ 
               ' -in ' + atlasname]
print('formulated commandline for transformix: ' , command_line_transformix)

# this step is really quick, normally 1min or so

formulated commandline for transformix:  ['transformix -out D:/AL066/ara2sample_atlas -tp D:/AL066/ara2sample_atlas/tx_transparam1.txt -in D:/AL066/atlas118-440.mhd']


## For sample 2 ara: calculate the inverse transform and transform the points


In [6]:
tempdir = fdialog.askdirectory(title='Please select a output directory')
param1=fdialog.askopenfile(initialdir=tempdir, title='select the first parameter file', initialdir = "D:/elastix_params_backup").name
param2=fdialog.askopenfile(initialdir=tempdir, title='select the second parameter file',initialdir = "D:/elastix_params_backup").name
points_name=fdialog.askopenfile(initialdir=tempdir, title='select the corresponding downsampled organized points').name

In [12]:
fixed_img=fdialog.askopenfile(initialdir=tempdir, title='select the fixed image').name
coefparam=fdialog.askopenfile(initialdir=tempdir, title='select the higherorder coeficcient file').name
#ie. here the higher order coefficient file is the transformparameters.1 (this is coupled with transformparameters.0 within the file) in the sample2ara folder

In [14]:
command_line= ['elastix -f '+ fixed_img+ ' -m ' + fixed_img+ 
               ' -out '+ tempdir+ ' -t0 '+ coefparam +' -p '+ param1+
              ' -p ' + param2]
print('formulated commandline: ' , command_line)


formulated commandline:  ['elastix -f D:/AL110/template_95to407.mhd -m D:/AL110/template_95to407.mhd -out D:/AL110/sample2ara_points -t0 D:/AL110/sample2ara/TransformParameters.1.txt -p D:/elastix_params_backup/sample2ara/01_ARA_affine.txt -p D:/elastix_params_backup/sample2ara/02_ARA_bspline.txt']


## change transformparam0 that is newly generated
Use newly generated transformparam file to transform the points

in transformparam0, change initial transform to NoInitialTransform

In [15]:
newparam=fdialog.askopenfile(initialdir=tempdir, title='select the newly generated transformparameter1 file').name

In [16]:
command_line= ['transformix -def '+ points_name+ ' -out '+ tempdir+ ' -tp '+ newparam ]
print('formulated commandline: ' , command_line)

formulated commandline:  ['transformix -def D:/AL110/AL110_25.0voxel_trace_1umStepsize.txt -out D:/AL110/sample2ara_points -tp D:/AL110/sample2ara_points/TransformParameters.1.txt']
